# Configure a Scenario for FloodAdapt

This notebook allows a user to configure a scenario to run using FloodAdapt. This includes a weather event, possible future climate or socio-economic projections, and adaptation measures. The user will be asked to provide input wherever necessay. The data needed to set up the weather event will be automatically fetched based on provided info.\
For both fetching the data and running the worklow, the right credentials are necessary. This is handled through EGI SSO, where the user should be a member of the dev.intertwin.eu VO

## **Step 0:** Import dependencies

In [ ]:
from pathlib import Path

from flood_adapt.objects.measures.measures import MeasureType

from DT_flood.utils.fa_scenario_utils import (
    create_event,
    create_measure,
    create_projection,
    create_scenario,
    create_strategy,
    get_database,
)
from DT_flood.utils.plot_utils import draw_database_map
from DT_flood.utils.plotting.fiat import list_agg_areas
from DT_flood.utils.workflow_utils import run_scenario


## **Step 1:** Load FloodAdapt database

In [ ]:
# Path to the FloodAdapt database for which we are configuring scenarios
fa_database = Path("/home/wotromp/InterTwin/workflow_refactor/fa_database/Humber_new")

database = get_database(database_path=fa_database)

## **Step 2:** Configure the weather Event

Here we will specify everything needed to setup an event. For now we will only consider scenarios which involve just a single event.

To fully specify an event we will need the following:
- A name for referencing
- Start and end time and dates

The relevant forcing data will be automatically fetched from the InterTwin datalake using rucio. This will default to using ERA5 data for all forcing variables for both SFINCS and WFLOW, except for the waterlevels at SFINCS' coastal boundary for which we use GTSM data. To use other data sources, please make sure the relevant data is on the datalake and provide the correct rucio scopes in dictionaries passed to the `sf_forcing` argument for SFINCS and `wf_forcing` for WFLOW.

Should you provide a name of an already existing event, no new event will be created and the existing one will be returned instead.

In [ ]:
# Get list of already existing events
events_existing = database.get_events()
print(events_existing['name'])

In [ ]:
# Name of event
event_name = 'test_event_rucio2'

# Start and end times of the event
start_time = '2013-12-04 00:00:00'
end_time = '2013-12-10 00:00:00'


In [ ]:
# Create event
event = create_event(database=database, name=event_name, start_time=start_time, end_time=end_time)

## **Step 3:** Configure projections

Here we specify, where applicable, the projected future changes relevant to our scenario. These come in two flavors: physical projections and socio-economic changes. The former contains changes on the hazard side (sea level rise, land subsidence etc.) while the latter contains changes to the impact side (population growth, economic growth etc.).\
The supported projections are:\
\
**Physical Projections:**
- Sea level rise
- Rainfall increase

**Socio-Economic Projections:**
- Population growth
- Economic growth

Should you provide a name of an already existing projection, no new projection will be created and the existing one will be returned instead.

In [ ]:
# Get list of already existing projections
proj_existing = database.get_projections()
print(proj_existing['name'])

In [ ]:
# Name of projection
projection_name = 'slr_test_pop_growth'

# Physical projections
sea_level_rise = 1.5
rainfall_increase = 0


# Socio-Economic projections
pop_growth = 10
economic_growth = 0

In [ ]:
# Create Projection
projection = create_projection(database=database, name=projection_name, sea_level_rise=sea_level_rise)

## **Step 4a:** Configure Measures

Here we configure all the measure we want to implement for this scenario. These come in either Impact or Hazard types, either reducing the vulnerability to a hazard, or dampen the hazard itself.\
The supported measures are:\
\
**Hazard Type:**
- Floodwall
- Pump
- Water square
- Green infrastructure
- Water storage

**Impact Type:**
- Elevate properties
- Floodproof properties
- Buyout properties

All measures need an area specification where they act. The function `draw_database_map()` below will open an interactive map where you can draw geometries to use as area selected. The selected area will be stored in the `selected_geom` variable. Impact measures can also act on an aggregation zone level. To use this, pass the `agg_area_name` argument with a name of an aggregation zone to the `draw_database_map()` function. You can then click on an aggregation zone to select it. If no geometry or area is selected, `selected_geom` will default to `all`, meaning the measure will either act on the entire domain (for Impact measures) or throw an error (for Hazard measures, since it is not a valid selection type)

Most measures also have a parameter describing the measure (height of a floodwall, elevation of newly elevated homes, etc.). The value of this parameter is to be set in the `measure_value` variable. The exceptions here are the Water storage, Greening, and Water Square measures. These require a list of 3 numbers describing in order the volume, height, and percentage area of the measure. See the [FloodAdapt documention](https://deltares-research.github.io/FloodAdapt/api_ref/objects/Measure.html) for more details.

Should you provide a name of an already existing meausure, no new measure will be created and the existing one will be returned instead.

In [ ]:
# print some useful stuff

print("Existing measures:")
meas_existing = database.get_measures()
print(meas_existing['name'])

print("Allowed measure types:")
print([e.value for e in MeasureType])

print("Existing Aggreagation area types:")
print(list_agg_areas(database=database))


In [ ]:
m, selected_geom = draw_database_map(database, agg_area_name="AdminZonesLevel3")
m

In [ ]:
# Double check selected geom/area
selected_geom

In [ ]:
# Set measure parameters
measure_name = "test_pump_estuary"
measure_type = "pump"

measure_value = 10.0
property_type = "commercial"

In [ ]:
# Create measure
measure = create_measure(database=database, name=measure_name, measure_type=measure_type, value=measure_value, selection=selected_geom, property_type=property_type)

In [ ]:
# Double check measure set up worked (validation of measure selection type only happens when trying to fetch it from database)
print("Existing measures:")
meas_existing = database.get_measures()
print(meas_existing['name'])

## **Step 4b:** Setup a strategy

Within a single scenario, multiple measures can be applied simultaneously when bundled together in a strategy. Here you can create a strategy by listing the measures you want to bundle in the `measure_list` variable.

Should you provide a name of an already existing strategy, no new strategy will be created and the existing one will be returned instead.

In [ ]:
print("Existing strategies:")
strat_existing = database.get_strategies()
print(strat_existing["name"])

In [ ]:
print("Existing measures:")
meas_existing = database.get_measures()
print(meas_existing['name'])

In [ ]:
# Set strategy parameters
strategy_name = "test_strategy_floodproof"
measure_list = ["test_floodproof"]

In [ ]:
# Create strategy
strategy = create_strategy(database=database, name=strategy_name, measure_list=measure_list)

## **Step 5:** Collect everything into a single Scenario

With events, projections, and strategies configured, we can now define a scenario to run. To do this, simply set the `scenario_event`, `scenario_strat`, and `scenario_proj` variables to the names of the desired event, strategy, and projection resp.

Should you provide a name of an already existing scenario, no new scenario will be created and the existing one will be returned instead.

In [ ]:
events_existing = database.get_events()
print(events_existing['name'])

In [ ]:
print("Existing strategies:")
strat_existing = database.get_strategies()
print(strat_existing["name"])

In [ ]:
proj_existing = database.get_projections()
print(proj_existing['name'])

In [ ]:
scen_existing = database.get_scenarios()
print(scen_existing["name"])

In [ ]:
# Set scenario parameters
scenario_name = "test_scenario_floodproof_only_pop_growth"
scenario_event = event.name # To use event just configured above, or name str of other pre-existing event
scenario_strat = strategy.name # To use strategy just configured above, or name str of other pre-existing strategy
scenario_proj = projection.name # To use projection just configured above, or name str of other pre-existing projection

In [ ]:
# Create scenario
scenario = create_scenario(database=database, name=scenario_name, event=scenario_event, strategy=scenario_strat, projection=scenario_proj)

## **Step 6:** Launch workflow

With a complete definition of the scenario we can run the workflow. To run the underlying models we will use OSCAR to trigger the relevant containers/services. For this to work we need to specify the oscar cluster where the services are defined in the `oscar_endpoint` variable, and provide an access token in `oscar_token`. For the latter we use refresh tokens from EGI SSO Check-in Portal.\
Other arguments can be set to configure the name of the OSCAR output folder (used internally during workflow execution), whether to offload the OSCAR services to HPC using InterLink, or enable caching of steps and verbose output to assist debugging.

In [ ]:
scen_existing = database.get_scenarios()
print(scen_existing["name"])

In [ ]:
scenario_name = "test_scenario_no_floodwall"

oscar_endpoint = 'https://oscar-ukri.intertwin.fedcloud.eu'
oscar_token = ""

In [ ]:
run_scenario(database=database, scenario_name=scenario_name, oscar_endpoint=oscar_endpoint, oscar_token=oscar_token, debug=True)

## **Step 7:** Plot results

With a succesfull run of a scenario completed, we can inspect and visualize the models and results from the scenario. The cells below will create interactive maps, one for each model, displaying both input and output data for each model. The plot markers are clickable, which will create a popup with graph or other information. 

In [ ]:
from DT_flood.utils.plot_utils import (
    draw_scenario_fiat,
    draw_scenario_ra2ce,
    draw_scenario_sfincs,
    draw_scenario_wflow,
)

scenario_name = "test_scenario_no_floodwall"

The cell below will create an interactive map for the SFINCS model. The argument `layer` currently can take as input either "dep" or "floodmap". This will create a static map of either the topobathy used by SFINCS or the floodmap it outputted. In addition, the river geometries used by SFINCS, and the points where forcing is set are plotted, both river discharge (dis points) and coastal waterlevels (bzs points). The forcing points are clickable, displaying a graph of the forcing timeseries at that point.

In [ ]:
draw_scenario_sfincs(database, scenario_name, layer="dep")

The cell below creates a map for the WFLOW model. First click on the button in the topright to select whether the warmup run or the event run is to be shown. The topography, river geometries, and reservoir geometries used in the model are shown as static plots. The plot markers are for either the points where a timeseries is outputted to use in SFINCS, or the downstream most point of a basin. Clicking on one of the points will create a popup showing the timeseries graph for the discharge at that point.

In [ ]:
draw_scenario_wflow(database, scenario_name)

This will create a map for the Delft-FIAT output. The `agg_layer` arguments takes as input either the name of an aggregation area used in the model, or "building_footprints". In the former case the map will display the building damages aggregated into the indicated areas, in the latter on an individual building level. *Note: doing this may take a long time for any reasonably sized area*. The floodmap used in the model is also overlayed.

In [ ]:
draw_scenario_fiat(database, scenario_name, agg_layer="gadm_level3")

The cell below opens an interactive map displaying the data from the RA2CE model on top of the floodmap for the scenario. The road network used is shown, with the damage to road sections indicated. The origin and destination points are plotted. Clicking on an origin point will show to route to the nearest destination point in both the base (non-flooded) case and the flooding case, together with a popup with additional information. *Note: in the case of a coarse/sparse roadnetwork, the routes may not be what is expected based on the plotted origin and destination points. This is because the points are plotted on their originally indicated position during model setup, not at the location at which they are snapped to the network during calculation.*

In [ ]:
draw_scenario_ra2ce(database, scenario_name)